# Using vireo for multiplexing, followed by lineage tracing of those groups

# 1. Set up

In [ ]:
#import os
#os.chdir("/data2/mito_lineage/")#Analysis/multiplex")

In [ ]:
# papermill --cwd /data2/mito_lineage -p INDIR {INDIR} -p OUTDIR data/processed/mttrace/CHIP_april08_2021_Croker/MTblacklist/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/clones -p  src/vireo/2_MT_Lineage_Construct.ipynb 

In [ ]:
pwd

### Remove this when running in snakemake

## 1.1 Parameters

In [ ]:
# INDIR=""
# OUTDIR=""
# N_DONORS=5

# INDIR= "data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/merged/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/multiplex"
# OUTDIR= "data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/merged/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/clones/"
# N_DONORS=2
DEBUG=True

INDIR= "data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/merged/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/multiplex"
OUTDIR = "data/processed/mttrace/TcellDupi_may17_2021/MTblacklist/merged/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/clones"
#N_DONORS=2
N_DONORS=4 #0 #Donor ID

nclones = 20
#n_clone_list="20,40,100"#[20, 40, 100]

In [ ]:
nclones = int(nclones)

In [ ]:
# n_clone_list = [int(x) for x in n_clone_list.split(",")]
# n_clone_list

In [ ]:
# # Parameters
# #INDIR="data/CHIP_april08_2021/MTblacklist/chrM/pseudo/minC200_minAF0.0001/numC25000_ispropFalse/"
# INDIR=""
# OUTDIR=""
# #OUTDIR="data/CHIP_april08_2021/MTblacklist/chrM/pseudo/minC200_minAF0.0001/numC25000_ispropFalse/lineages"
# N_DONORS=5

## 1.2 Import packages

In [ ]:
from os.path import join, exists, dirname
from icecream import ic
from glob import glob
import mplh.cluster_help as ch
import os
from vireoSNP.plot.base_plot import heat_matrix
from vireoSNP import Vireo
import src.pseudo_batch as pb
import vireoSNP
ic(vireoSNP.__version__)
from tqdm import tqdm
import numpy as np
from scipy import sparse
from scipy.io import mmread
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from src.vireo.vireo_utils import run_vireo, plot_vireo_out, extract_clusters, run_elbo, separate_donors
%load_ext autoreload
%autoreload 2
from src.vireo.vireo_utils import run_lineage
#np.set_icoptions(formatter={'float': lambda x: format(x, '.5f')})

In [ ]:
if not DEBUG:
    ic.disable()


# 2 Lineage construction for each donor

In [ ]:
# num_cores = 12# N_DONORS #multiprocessing.cpu_count()
# inputs = tqdm(np.arange(N_DONORS))
# for n in np.arange(N_DONORS):
#     ic('n', n)
#     run_lineage(n, INDIR, OUTDIR, n_clone_list=[5, 10, 20, 40, 100], rerun_model=False)

In [ ]:
cells_meta = pd.read_csv(join(INDIR, f"cells_meta.tsv"), sep='\t')

cells_meta=cells_meta.astype({"donor_index": "Int64", "donor": "Int64"})

sns.countplot(cells_meta["donor"].fillna(-1))

In [ ]:
def run_lineage_wrap(indir, outdir, n_donors, nclones, cells_meta):
 #   for clone in n_clone_list:
#     cells_meta_col = f"lineage{nclones}"
#     curr_outdir = join(outdir, cells_meta_col)

    if not exists(outdir):
        os.mkdir(outdir)
    curr_cells_meta = cells_meta.copy()
    all_cells_meta = [] #concat across donors

    for donor in np.arange(n_donors):
        print(f'clone: {nclones} donor: {donor}')
        curr_ad_f = join(indir, f"donor{donor}.AD.mtx")
        curr_dp_f = join(indir, f"donor{donor}.DP.mtx")
        curr_labels_f = join(indir, f"donor{donor}.labels.txt")
        curr_vcf_f = join(indir, f"donor{donor}.vcf")
        curr_labels_rawID = join(indir, f"cell_labels.donor{donor}.txt")
        prefix=f"donor{donor}"

        ic(curr_ad_f)
        ic(curr_dp_f)
        if not ((cells_meta["donor"] == donor).any()):
            print(f"Donor {donor} is empty. Continuing")
            continue
        curr_ad = mmread(curr_ad_f).tocsc()
        curr_dp = mmread(curr_dp_f).tocsc()
        curr_labels = pd.read_csv(curr_labels_f, index_col=0)

        curr_vcf = pd.read_csv(curr_vcf_f, sep='\t')
        ic('curr_vcf')
        ic(curr_vcf.head())
        #curr_cells_meta = pd.read_csv(curr_labels_rawID, index_col=0)
#             ic('curr_labels')
#             ic(curr_labels.head())
        #prefix = f"donor{donor}_"
        model_name = f"{prefix}"#lineage_{nclones}"
        modelCA, elbo = run_vireo(curr_ad, curr_dp, nclones, n_cores=32, plot_qc=True,out_f=join(outdir, model_name))
        try:
            doublet_prob = modelCA.predict_doublet(curr_ad, curr_dp, update_GT=False, update_ID=False)[0].sum(axis=1)
        except AttributeError: # New version of Vireo 2021
            #doublet_prob = vireoSNP.utils.vireo_doublet.predict_doublet(modelCA,AD, DP, update_GT=False, update_ID=False)[0].sum(axis=1)
            doublet_prob = vireoSNP.utils.vireo_doublet.predict_doublet(modelCA, curr_ad, curr_dp, update_GT=False, update_ID=False)[0].sum(axis=1)

        separate_donors(curr_ad, curr_dp, modelCA, curr_cells_meta, outdir,
                        doublet_prob,
                        prob_thresh=0.9, doublet_thresh=0.9,
                        cells_ind_col='donor_index',
                        out_name="lineage", prefix=prefix, cells_filt_col="donor", 
                        cells_filt_val=donor, vars_meta=curr_vcf)    

        lin_cells_meta = pd.read_csv(join(outdir, f"{prefix}cells_meta.tsv"), sep='\t')
        ic(lin_cells_meta.head())
        ic(lin_cells_meta.shape)
        all_cells_meta.append(lin_cells_meta)
        ic("Plotting AF SNPs")
        lin_cells_meta = lin_cells_meta.astype({"lineage": 'Int64'}).fillna(-1).astype(str)
        plot_vireo_out(modelCA, out_f=join(outdir, model_name), 
                       labels=lin_cells_meta[["condition", "lineage"]], 
                       doublet_prob=doublet_prob)

        plt.close()

    # Concat cells_meta across donors
    all_cells_meta = pd.concat(all_cells_meta, axis=0)
    all_cells_meta = all_cells_meta.astype({"lineage_index": "Int64", "lineage": "Int64"})
    ic(all_cells_meta.head())
    ic(all_cells_meta.shape)
    all_cells_meta.to_csv(join(outdir, "cells_meta.tsv"), sep='\t')
    return

In [ ]:
run_lineage_wrap(INDIR, OUTDIR, N_DONORS, nclones, cells_meta)